<div dir="rtl" style="width:90%; margin:auto;">
<h3>حافظه مدیریت‌شده (Managed Memory)</h3>
<p><strong>مدیریت خودکار توسط سیستم زمان‌اجرا</strong>:</p>
<ul>
<li>در محیط‌های مدیریت‌شده مانند .NET، مدیریت حافظه به عهده سیستم زمان‌اجرا (CLR) است.</li>
<li>جمع‌آوری زباله (Garbage Collection یا GC) به صورت خودکار اشیاء بدون استفاده را شناسایی و حافظه آن‌ها را آزاد می‌کند.</li>
</ul>

<h3>حافظه مدیریت‌نشده (Unmanaged Memory)</h3>
<p><strong>مدیریت دستی توسط برنامه‌نویس</strong>:</p>
<ul><li>در زبان‌های برنامه‌نویسی غیرمدیریت‌شده مانند C++، مدیریت حافظه به عهده برنامه‌نویس است.</li><li>برنامه‌نویس باید حافظه را به صورت دستی تخصیص دهد و آزاد کند (معمولاً با استفاده از کلمات کلیدی <code>malloc</code> و <code>free</code> در C یا <code>new</code> و <code>delete</code> در C++).</li></ul>

</div>

<div dir="rtl" style="width:90%; margin:auto;">
<h3>نشت حافظه مدیریت‌شده (Managed Memory Leaks)</h3>
<p>در زبان‌های برنامه‌نویسی غیرمدیریت‌شده مانند C++، برنامه‌نویسان باید به صورت دستی حافظه‌ای را که دیگر نیازی به آن نیست، آزاد کنند؛ در غیر این صورت، نشت حافظه رخ خواهد داد. در دنیای مدیریت‌شده، این نوع خطاها به دلیل سیستم جمع‌آوری زباله (GC) خودکار CLR امکان‌پذیر نیست.</p>

<h4>نشت حافظه در برنامه‌های مدیریت‌شده</h4>
<p>با این حال، برنامه‌های بزرگ و پیچیده .NET می‌توانند شکل خفیف‌تری از همین مشکل را با همان نتیجه نهایی نشان دهند: برنامه به تدریج حافظه بیشتری مصرف می‌کند تا زمانی که در نهایت نیاز به راه‌اندازی مجدد دارد. خبر خوب این است که نشت حافظه مدیریت‌شده معمولاً آسان‌تر تشخیص داده و پیشگیری می‌شود.</p>

<h4>علت نشت حافظه مدیریت‌شده</h4>

<p>نشت حافظه مدیریت‌شده به دلیل زنده ماندن اشیاء بدون استفاده به واسطه مراجع فراموش‌شده یا بدون استفاده ایجاد می‌شود. یک نمونه رایج، <strong>مدیریت رویدادها</strong> (event handlers) است؛ این‌ها یک مرجع به شیء هدف نگه می‌دارند (مگر اینکه هدف یک متد استاتیک باشد).</p>

<h3>چگونه نشت حافظه مدیریت‌شده رخ می‌دهد</h3>
<ul><li><strong>مراجع فراموش‌شده</strong>: اگر شما مرجع به یک شیء را فراموش کنید و این مرجع همچنان باقی بماند، شیء مربوطه توسط GC جمع‌آوری نمی‌شود و در حافظه باقی می‌ماند.</li><li><strong>مدیریت رویدادها</strong>: اگر یک شیء برای مدیریت یک رویداد ثبت‌نام کند و پس از آن فراموش شود که از رویداد لغو ثبت‌نام شود، مرجع به آن شیء باقی می‌ماند و حافظه آن آزاد نمی‌شود.</li></ul>

<h3>تشخیص و پیشگیری</h3>

<ul><li><strong>لغو ثبت‌نام رویدادها</strong>: مطمئن شوید که شیء‌هایی که برای مدیریت رویدادها ثبت‌نام می‌کنند، به موقع از رویداد لغو ثبت‌نام می‌شوند.</li><li><strong>استفاده از ابزارهای تشخیص نشت حافظه</strong>: از ابزارهای موجود برای تشخیص و تحلیل نشت حافظه استفاده کنید. این ابزارها می‌توانند به شما کمک کنند تا مراجع فراموش‌شده و اشیاء زنده بدون استفاده را شناسایی کنید.</li><li><strong>بررسی دوره‌ای کد</strong>: کد خود را به طور دوره‌ای بررسی کنید و از اینکه مرجع‌های غیرضروری و بدون استفاده در کد شما باقی نمانده‌اند، اطمینان حاصل کنید.</li></ul>
</div>

In [ ]:
class Host
{
    public event EventHandler Click;
}
class Client
{
    Host _host;
    public Client (Host host)
    {
        _host = host;
        _host.Click += HostClicked;
    }
    void HostClicked (object sender, EventArgs e) {  }
}

//The following test class contains a method that instantiates 1,000 clients:
class Test
{
    static Host _host = new Host();
    public static void CreateClients()
    {
        Client[] clients = Enumerable.Range (0, 1000)
            .Select (i => new Client (_host))
            .ToArray();
        // Do something with clients ...

        /*
        بعد از اجرای این کد باید کلاینت ها جمع آوری شوند
        این کار انجام نمی شود، چون در یک رویداد مشترک شده اند و زنده می مانند.
        */
    }
}

One way to solve this is to make ***Client*** `implement IDisposable` and, in the
Dispose method, unhook the event handler:

In [ ]:
class Host
{
    public event EventHandler Click;
}
class Client : IDisposable
{
    Host _host;
    public Client (Host host)
    {
        _host = host;
        _host.Click += HostClicked;
    }
    void HostClicked (object sender, EventArgs e) {  }

    public void Dispose() { _host.Click -= HostClicked; }
}

class Test
{
    static Host _host = new Host();
    public static void CreateClients()
    {
        Client[] clients = Enumerable.Range (0, 1000)
            .Select (i => new Client (_host))
            .ToArray();
        // Do something with clients ...

        Array.ForEach (clients, c => c.Dispose());
    }
}

### Timers

<div dir="rtl" style="width:90%; margin:auto;">
<h4>تایمر در فضای نام System.Timers</h4>
<p>تایمرهای <code>System.Timers</code> به گونه‌ای طراحی شده‌اند که در فواصل زمانی مشخص کدهایی را اجرا کنند. این تایمرها می‌توانند به نشت حافظه منجر شوند اگر:</p>
<ol><li><strong>مدیریت نشوند</strong>: تایمرها همچنان فعال بمانند حتی وقتی که دیگر به آن‌ها نیاز نیست. این به این معناست که تایمرها همچنان به رویدادهای زمان‌بندی شده خود ادامه می‌دهند و مرجعی به شیء هدف نگه می‌دارند.</li><li><strong>لغو نشوند</strong>: تایمرها به درستی لغو (dispose) نشوند، که باعث می‌شود مراجع به اشیاء هدف باقی بماند و حافظه آن‌ها آزاد نشود.</li></ol>

<h3>جلوگیری از نشت حافظه در تایمرها</h3>
<p>برای جلوگیری از نشت حافظه ناشی از تایمرها، می‌توان اقدامات زیر را انجام داد:</p>

<p><strong>لغو کردن تایمر</strong>: وقتی که دیگر به تایمر نیازی نیست، باید به درستی آن را لغو (dispose) کنید.</p>



</div>

In [ ]:
timer.Stop();
timer.Dispose();

<div dir="rtl" style="width:90%; margin:auto;">
<p><strong>استفاده از بلوک‌های Using</strong>: می‌توانید از بلوک‌های <code>using</code> برای مدیریت تایمرها استفاده کنید تا مطمئن شوید که به درستی آزاد می‌شوند.</p>
</div>

In [ ]:
using (var timer = new System.Timers.Timer())
{
    // تنظیم تایمر
}


<div dir="rtl" style="width:90%; margin:auto;">
<p><strong>توجه به رویدادها:</strong>: اگر تایمرها رویدادهایی را مدیریت می‌کنند، باید اطمینان حاصل کنید که این رویدادها به درستی لغو می‌شوند تا مراجع باقی نمانند.</p>
</div>

In [ ]:
using System.Timers;
class Foo
{
    Timer _timer;
    Foo()
    {
        _timer = new System.Timers.Timer { Interval = 1000 };
        _timer.Elapsed += tmr_Elapsed;
        _timer.Start();
    }
    void tmr_Elapsed (object sender, ElapsedEventArgs e) { }
}

//Unfortunately, instances of Foo can never be garbage collected!

In [ ]:
using System.Timers;
class Foo : IDisposable
{
    Timer _timer;
    Foo()
    {
        _timer = new System.Timers.Timer { Interval = 1000 };
        _timer.Elapsed += tmr_Elapsed;
        _timer.Start();
    }
    void tmr_Elapsed (object sender, ElapsedEventArgs e) { }

    public void Dispose() { _timer.Dispose(); }
}


<div dir="rtl" style="width:90%; margin:auto;">
<h3>تایمر در فضای نام System.Threading</h3>
<p>.NET مراجع به تایمرهای threading فعال را نگه نمی‌دارد.
در عوض، مستقیماً به callback delegates (نمایندگان فراخوانی) اشاره می‌کند.
</p>
<h4>مدیریت خودکار تایمرهای System.Threading</h4>
<p>این ویژگی به این معناست که اگر شما فراموش کنید یک تایمر threading را لغو (dispose) کنید، یک finalizer به طور خودکار فعال می‌شود که تایمر را متوقف و لغو می‌کند. این امر به جلوگیری از نشت حافظه کمک می‌کند.</p>
</div>

In [ ]:
var tmr = new System.Threading.Timer (TimerTick, null, 1000, 1000);

GC.Collect(); 
// the timer will be collected and finalized before it has a chance to fire
//even once

System.Threading.Thread.Sleep (10000); // Wait 10 seconds


static void TimerTick (object notUsed) { Console.WriteLine ("tick"); }

In [ ]:
//we can fix this by disposing of the timer when we’re done with it

using (var tmr = new System.Threading.Timer (TimerTick, null, 1000, 1000))
{
    GC.Collect();
    System.Threading.Thread.Sleep (10000); // Wait 10 seconds
}

static void TimerTick (object notUsed) { Console.WriteLine ("tick"); }

### Diagnosing Memory Leaks

<div dir="rtl" style="width:90%; margin:auto;">
<h4>پایش مصرف حافظه</h4>
<p>ساده‌ترین راه برای جلوگیری از نشت حافظه مدیریت‌شده، پایش مصرف حافظه به صورت پیشگیرانه هنگام نوشتن برنامه است. شما می‌توانید مصرف حافظه فعلی اشیاء یک برنامه را به صورت زیر بدست آورید (آرگومان <code>true</code> به GC می‌گوید که ابتدا یک جمع‌آوری زباله انجام دهد):</p>
</div>

In [ ]:
long memoryUsed = GC.GetTotalMemory(true);

<div dir="rtl" style="width:90%; margin:auto;">
<h4>ابزارهای تشخیص نشت حافظه</h4>
<p>اگر شما یک برنامه بزرگ دارید که دچار نشت حافظه مدیریت‌شده است، ابزار <code>windbg.exe</code> می‌تواند به شما در پیدا کردن آن کمک کند. همچنین ابزارهای گرافیکی کاربرپسندتری وجود دارند مانند:</p>
<ul><li><strong>CLR Profiler</strong>: ابزار مایکروسافت برای پروفایلینگ حافظه در CLR.</li><li><strong>SciTech Memory Profiler</strong>: یک پروفایلر حافظه که به شما در تشخیص و تحلیل نشت‌های حافظه کمک می‌کند.</li><li><strong>ANTS Memory Profiler</strong>: پروفایلر حافظه از شرکت Red Gate که ابزار گرافیکی قدرتمندی برای تحلیل مصرف حافظه ارائه می‌دهد.</li></ul>
</div>